In [23]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json


In [14]:
train_data_dir = 'data/task1/train'
validation_data_dir = 'data/task1/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 20
batch_size = 16
img_width, img_height = 150,150

In [15]:
def simple_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy', precision, recall, f1_score])
    return model


In [11]:
model = simple_model()

In [13]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [6]:

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [7]:

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 277 images belonging to 2 classes.


In [8]:

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 32 images belonging to 2 classes.


In [9]:
def f1_score(true, pred):
    p = precision(true, pred)
    r = recall(true, pred)
    return 2 * (p * r) / (p + r + 1e-6)

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [16]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/20
125/125 [==============================] - 43s 346ms/step - loss: 0.1089 - acc: 0.9599 - precision: 0.9667 - recall: 0.9846 - f1_score: 0.9745 - val_loss: 0.5479 - val_acc: 0.8438 - val_precision: 0.8842 - val_recall: 0.9198 - val_f1_score: 0.9006
Epoch 2/20
125/125 [==============================] - 45s 359ms/step - loss: 0.1174 - acc: 0.9645 - precision: 0.9704 - recall: 0.9868 - f1_score: 0.9775 - val_loss: 0.2717 - val_acc: 0.8750 - val_precision: 0.9203 - val_recall: 0.9207 - val_f1_score: 0.9191
Epoch 3/20
125/125 [==============================] - 38s 302ms/step - loss: 0.1096 - acc: 0.9610 - precision: 0.9682 - recall: 0.9826 - f1_score: 0.9742 - val_loss: 0.6441 - val_acc: 0.8438 - val_precision: 0.8848 - val_recall: 0.9204 - val_f1_score: 0.9001
Epoch 4/20
125/125 [==============================] - 38s 305ms/step - loss: 0.0927 - acc: 0.9630 - precision: 0.9723 - recall: 0.9831 - f1_score: 0.9768 - val_loss: 0.3285 - val_acc: 0.8750 - val_precision: 0.9203 - val_re

In [19]:
def save(name):# serialize model to JSON
    model_json = model.to_json()
    with open(name+'.json', "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(name+".h5")
    print("Saved model to disk")
# late

In [20]:
def load(name):
    # load json and create model
    json_file = open(name+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(name+'.h5')
    print("Loaded model from disk")
    return loaded_model


In [21]:
save('model1')

Saved model to disk


In [24]:
m = load('model1')

Loaded model from disk


In [25]:
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
__________